In [99]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="601f4351-33bb-4d76-96ca-886940409b3d")
resource_group = os.getenv("RESOURCE_GROUP", default="mlopcent-AML-RG")
workspace_name = os.getenv("WORKSPACE_NAME", default="mlopcent-AML-WS")
workspace_region = os.getenv("WORKSPACE_REGION", default="centralus")

from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = False,
                      sku = 'basic',
                      exist_ok = True)
ws.get_details()

{'id': '/subscriptions/601f4351-33bb-4d76-96ca-886940409b3d/resourceGroups/mlopcent-AML-RG/providers/Microsoft.MachineLearningServices/workspaces/mlopcent-AML-WS',
 'name': 'mlopcent-AML-WS',
 'location': 'centralus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Enterprise',
 'workspaceid': '9f71fb47-35b4-4cde-9787-6879d312a8e1',
 'description': '',
 'friendlyName': 'mlopcent-AML-WS',
 'creationTime': '2019-11-12T01:09:07.4987878+00:00',
 'containerRegistry': '/subscriptions/601f4351-33bb-4d76-96ca-886940409b3d/resourcegroups/mlopcent-aml-rg/providers/microsoft.containerregistry/registries/mlopcentamlcr',
 'keyVault': '/subscriptions/601f4351-33bb-4d76-96ca-886940409b3d/resourcegroups/mlopcent-aml-rg/providers/microsoft.keyvault/vaults/mlopcent-aml-kv',
 'applicationInsights': '/subscriptions/601f4351-33bb-4d76-96ca-886940409b3d/resourcegroups/mlopcent-aml-rg/providers/microsoft.insights/components/mlopcent-aml-ai',
 'identityPrincipalId': '159e1229-ee0a-44ad-9432-

In [100]:
from azureml.core import Datastore
blob_datastore_name='isic2018' # Name of the Datastore  to workspace
container_name=os.getenv("BLOB_CONTAINER", "isic2018") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "mlblobdatastore") # Storage account name
account_key=os.getenv("BLOB_ACCOUNT_KEY", "bPlInBOqf0kfPpSNYeemRKNiOfcWsMWAUfR3ieyTUpxBKn/FEkZG9RgHUQfVjNtI3ky32wZ+LrjCe/oVC9M2eg==") # Storage account key

blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

In [101]:
from azureml.core import Datastore
#get named datastore from current workspace
datastore = Datastore.get(ws, datastore_name='isic2018')

In [102]:
from azureml.core.dataset import Dataset


datastore_paths = [(datastore, 'ISIC2018_Task1-2_Training_Input')]

isic_ds_training = Dataset.File.from_files(path=datastore_paths)

In [103]:
# create a new version of titanic_ds
isic_ds = isic_ds_training.register(workspace = ws,
                                 name = 'isic_ds',
                                 description = 'isic training data',
                                 create_new_version = True)

In [104]:
from azureml.core.dataset import Dataset

isic_ds_name = 'isic_inference_data'

dataset = Dataset.get_by_name(ws, 'isic_ds')
named_isic_ds = dataset.as_named_input(isic_ds_name)

In [79]:
# list the files referenced by mnist dataset
dataset.to_path()

array(['/ISIC_0000000.jpg', '/ISIC_0000001.jpg', '/ISIC_0000003.jpg', ...,
       '/ISIC_0016070.jpg', '/ISIC_0016071.jpg', '/ISIC_0016072.jpg'],
      dtype=object)

In [105]:
from azureml.pipeline.core import Pipeline, PipelineData
output_dir = PipelineData(name="inferences_horovod", 
                          datastore=datastore, 
                          output_path_on_compute="ISIC2012_MaskRCNN_Horovod_Inference")

In [106]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'cpucluster')
compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MAX_NODES', 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'STANDARD_D2_V2')


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes, 
                                                                max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. cpucluster


In [82]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name='mask_rcnn_hvd_finetune')

In [16]:
from azureml.core import Run
run_id = 'mask_rcnn_hvd_finetune_1577083350_0ceed65a'
fetched_run = Run(exp, run_id)
fetched_run

Experiment,Id,Type,Status,Details Page,Docs Page
mask_rcnn_hvd_finetune,mask_rcnn_hvd_finetune_1577083350_0ceed65a,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
model_dir = 'horovod_model'
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

In [19]:
#fetched_run.get_file_names()
fetched_run.download_file('outputs/lesions_logs/lesion20191223T0644/mask_rcnn_lesion_0020.h5', model_dir)

In [107]:
from azureml.core.model import Model

# Register the downloaded model 
model = Model.register(model_path="horovod_model/",
                       model_name="mask_rcnn_horovod",
                       tags={'model': "Mask_RCNN_Horovod",'dataset': "ISIC_Lesion"},
                       description="Mask_RCNN Keras model trained with Horovod",
                       workspace=ws)

Registering model mask_rcnn_horovod


In [108]:
import os

scripts_folder = "mask_rcnn_horovod_inference"
script_file = "lesions_inference.py"

# peek at contents
with open(os.path.join(scripts_folder, script_file)) as inference_file:
    print(inference_file.read())


# import the necessary packages
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import utils
import numpy as np
import imutils
import cv2
import os
from azureml.core import Model
import argparse




class LesionBoundaryInferenceConfig():
	# set the number of GPUs and images per GPU (which may be
	# different values than the ones used for training)
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

	# set the minimum detection confidence (used to prune out false
	# positive detections)
	DETECTION_MIN_CONFIDENCE = 0.9


def init():

	global model,output_dir

	# construct the argument parser and parse the arguments
	ap = argparse.ArgumentParser()
	ap.add_argument('--output_dir', type=str, dest='output_dir', help='output dir')

	args = vars(ap.parse_args())



# create output directory if it does not exist

	output_dir = args["output_dir"]
	os.makedirs(output_dir, exist_ok=True)


	LOGS_AND_MODEL_DIR = "./outputs/lesions_logs"
	os.makedirs(LOGS_AND_MODEL_DIR, exist_ok=True

In [109]:
from azureml.core import Environment
from azureml.core.runconfig import CondaDependencies, DEFAULT_CPU_IMAGE

batch_conda_deps = CondaDependencies.create(pip_packages=['scikit-image','cython','Pillow','numpy','azureml-sdk','tensorflow','keras','azureml-dataprep[pandas,fuse]','imutils','opencv-python','h5py'])

batch_env = Environment(name="batch_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE

In [111]:
from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    source_directory=scripts_folder,
    entry_script=script_file,
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=compute_target,
    node_count=2,
    logging_level="DEBUG")

In [112]:
from azureml.contrib.pipeline.steps import ParallelRunStep

parallelrun_step = ParallelRunStep(
    name="batch-lesion-horovod",
    models=[model],
    parallel_run_config=parallel_run_config,
    inputs=[named_isic_ds],
    output=output_dir,
    arguments=["--output_dir", output_dir],
    allow_reuse=False
    
)

In [113]:
from azureml.pipeline.core import Pipeline
from azureml.core.experiment import Experiment

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'batch_inference_test').submit(pipeline)

Created step batch-lesion-horovod [edcee79d][c7191358-f868-4bc8-ae69-bef4b7009d06], (This step will run and generate new outputs)
Using data reference isic_inference_data_0 for StepId [f09e831a][7ddbde14-8e1c-4980-80a3-f5c45aea1aad], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun e775a8bb-2b16-4155-9a39-96c2db5fa671
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/batch_inference_test/runs/e775a8bb-2b16-4155-9a39-96c2db5fa671?wsid=/subscriptions/601f4351-33bb-4d76-96ca-886940409b3d/resourcegroups/mlopcent-AML-RG/workspaces/mlopcent-AML-WS


In [90]:
pipeline_run.cancel()

In [89]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()



_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [ ]:
pipeline_run.wait_for_completion(show_output=True)